# Getting data

In [ ]:
import numpy as np
import data
import matplotlib.pyplot as plt
import warnings

In [ ]:
x, y = data.generate_gp(nb=50, rng=[-10, 10], x0=[-9, -5, 1, 7, 11], noise=0.0, w=0.01, scale=10)

In [ ]:
plt.plot(x,y, 'bo')

# Fitting a GP to data using GPFlow

In [ ]:
import gpflow 

## Getting, viewing and modifying model

### Getting kernel
GPFlow provides multiple kernels: Linear, Matern 12, 32, 53, Exponential, Squared Exponential, Cosine ... 
It is also possible to obtain new kernels by combining them, or creating a new one. 

### Modifying parameters
It is possible to modify parameters, apply constraints through transforms, fix some parameters, apply priors to the variance, etc .. This should be done before compiling the model. 

In [ ]:
warnings.simplefilter('ignore')
# meanf = gpflow.mean_functions.Linear()
k = gpflow.kernels.Matern32(1, variance=1, lengthscales=1.2)
m = gpflow.models.gpr.GPR(x, y, kern=k, mean_function=None)
m
m.kern.lengthscales = 0.5
m.likelihood.variance = 0.01
# m.kern.lengthscales.transform = gpflow.transforms.Exp()
# m.kern.variance.trainable = False
# m.kern.variance.prior = gpflow.priors.Gamma(2,3)

## Optimization and plotting results

### Optimizers: 
GPFlow provides many optimizers: 
- ScipyOptimizer: Scipy minimize function
- AdamOptimizer, AdadeltaOptmizer, ..: Based on TensorFlow 

In [ ]:
warnings.simplefilter('ignore')
opt = gpflow.train.ScipyOptimizer()
m.compile()
opt.minimize(model=m)

In [ ]:
m

In [ ]:
x_new = np.linspace(-10, 10, 100).reshape(100, 1)
mean, var = m.predict_y(x_new)
samples = m.predict_f_samples(x_new, 5)
plt.plot(x, y, 'ro', mew=0.5)
line, = plt.plot(x_new, mean, 'C0', lw=2)
plt.plot(x_new, samples[:, :, 0].T, 'C0', linewidth=.5)
_ = plt.fill_between(x_new[:,0], mean[:,0] - 2*np.sqrt(var[:,0]), mean[:,0] + 2*np.sqrt(var[:,0]), color=line.get_color(), alpha=0.2)